# GlassFlow Pipeline with Postgres to Clickhouse
This example shows how to create a pipeline with postgres CDC as a data source and clickhouse as a data sink. Both of these are available as a managed connector in GlassFlow

In [ ]:
%pip install "glassflow>=2.0.8"

In [ ]:
import glassflow

In [ ]:
# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = ""


## postgres configuration  
Please see docs at https://www.glassflow.dev/docs/integrations/sources/postgres-cdc on how to setup the parameters

In [ ]:

# postgres credentials  
# Please see docs at https://www.glassflow.dev/docs/integrations/sources/postgres-cdc on how to setup the parameters
POSTGRES_HOST = ""
POSTGRES_PORT = ""
POSTGRES_USER = ""
POSTGRES_PASSWORD = ""
POSTGRES_DB_NAME = ""
REPLICATION_SLOT = ""
REPLICATION_PLUGIN_ARGS = ""

## Clickhouse configuration 
Please see docs at https://www.glassflow.dev/docs/integrations/sinks/clickhouse on how to setup the parameters

In [ ]:
# Clickhouse configuration 
CLICKHOUSE_ADDRESS = ""
CLICKHOUSE_DATABASE = "" 
CLICKHOUSE_USERNAME = ""
CLICKHOUSE_PASSWORD = ""
CLICKHOUSE_TABLE = ""

In [ ]:
client = glassflow.GlassFlowClient(
    personal_access_token=personal_access_token
)

In [ ]:
# Get the space named "examples" (or create one if no space is found)
list_spaces = client.list_spaces()

space_name = "examples"
for s in list_spaces.spaces:
    if s["name"] == space_name:
        space = glassflow.Space(
            personal_access_token=client.personal_access_token,
            id=s["id"], 
            name=s["name"]
        )
        break
else:
    space = client.create_space(name=space_name)

print(f"Space \"{space.name}\" with ID: {space.id}")

# Setup Pipeline 

In [ ]:
pipeline_name = "postgres-to-clickhouse-example"
transformation_file='transform.py' # local file where transformation code exists
space_id = space.id

In [ ]:
pipeline = client.create_pipeline(
    name=pipeline_name, 
    transformation_file=transformation_file,
    space_id=space_id, 
    source_kind="postgres",
    source_config={
        "db_host": POSTGRES_HOST,
        "db_port": POSTGRES_PORT,
        "db_user": POSTGRES_USER,
        "db_pass": POSTGRES_PASSWORD,
        "db_name": POSTGRES_DB_NAME,
        "replication_slot": REPLICATION_SLOT,
        "replication_output_plugin_args": REPLICATION_PLUGIN_ARGS
    },
    sink_kind="clickhouse",
    sink_config={
        "addr": CLICKHOUSE_ADDRESS,
        "database": CLICKHOUSE_DATABASE,
        "username": CLICKHOUSE_USERNAME,
        "password": CLICKHOUSE_PASSWORD,
        "table": CLICKHOUSE_TABLE,
    }
)
print("Pipeline ID:", pipeline.id)